# Latihan-3a Basic Gassmann

Latihan ini fokus terhadap diskusi Gassmann sebagai salahsatu alurkerja inti estimasi shear velocity, pemodelan, perturbasi properti yang akan sering dilakukan pada latihan-latihan selanjutnya
<br><br> _citation for this page: "Adi Widyantoro, 2021, Pertamina Seismic QI Course, Latihan-3a Basic Gassmann.ipynb, accessed MM DD, YYYY."_  
<br>
>(update terakhir tanggal 1 Juli 2021 oleh Adi Widyantoro)
<hr>

__Tahap-1__ 
<br> Tetap menggunakan pre-built libraries dasar seperti numpy, matplotlib dan pandas. Serta tambahan fungsi yang telah dibuat untuk latihan sebelumnya
<br> 
        
        special notes_: brugeslibrary adalah kumpulan pre-defined functions oleh Agile Geoscience bruges:
        https://github.com/agile-geoscience/bruges
        
        The bruges library's credits go to contributors: Evan Bianco, Ben Bougher, Matt Hall, Alessandro Amato del Monte, 
        Wes Hamlyn, Sean Ross-Ross

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import math
from brugeslibrary import backus
mpl.style.use('default') # default, seaborn

__Tahap-2__
<br>Tetap menggunakan data sumur yang sama untuk melakukan analisis pada real samples

In [ ]:
# adi2=pd.read_csv('/content/adi02logs.csv') #jika menggunakan google colab' local
adi02=pd.read_csv('adi02logs.csv', warn_bad_lines=True, index_col=0)
adi02.rename(columns={'Depth_m':'Depth', 'Vp_km/s':'Vp', 'Vs_km/s':'Vs','GammaRay_api':'GR',
                      'Density_g/cc':'Rho','Caliper_m':'Cal','Swt_v/v':'Swt','Phit_v/v':'Phit', 
                      'Vsh_v/v':'Vsh'}, inplace=True)

In [ ]:
# adi02tops=pd.read_csv('/content/adi02markers.csv') #jika menggunakan google colab' local
adi02tops=pd.read_csv('adi02markers.csv')
adi02tops.rename(columns={'Tops':'marker','Depth_m':'markerdep'}, inplace=True)
marker=np.array(adi02tops['marker'])
markerdep=np.array(adi02tops['markerdep'])

__Tahap-3__
<br>Refresh fungsi dasar dan konstanta yang digunakan

#### Basic Elastic
<br>Signifikansi hubungan elastic properties dasar yang akan didiskusikan, perlu dipahami dan akan digunakan berulang dalam gassmann
Hubungan moduli dan velocities:

<font size=4>$V_p = \sqrt{{M} / {\rho}}$; $M = K+4/3\mu$ 
<br><br> $V_s = \sqrt{\mu /{\rho}}$</font>

K adalah _Bulk Modulus_, M adalah ekspresi _Compressional Modulus_, dan $\mu$ adalah Shear Modulus

Hubungan dasar ini memberikan dasar perhitungan K dan $\mu$ dalam gassmann ika velocity diketahui, sehingga dapat dituliskan kembali sbb: 
<br><br>
<font size=4>$\mu = V_s^2.\rho$</font>, dan <font size=4>$K = V_p^2.\rho - 4/3\mu$</font>, atau <font size=4>$K=\rho.(V_p^2 - 4/3 V_s^2)$</font>

In [ ]:
"---jika vp, vs dan bulk densitas rho diketahui:---"
vp1 = 2.4; rho1 = 2.65; vs1=1.2

"---maka mu dapat dihitung---"
mu1 = rho1*vs1**2
K1 = rho1*(vp1**2 - (4/3)*mu1)

print('K=%.2f'%K1, 'mu=%.2f'%mu1)

In [ ]:
"---jika K, mu dan bulk densitas rho diketahui:---"
K2=8.3; mu2=5.92; rho2=2.78

"---maka vp,vs baru dapat dihitung ulang:---"
M2 = K2 + (4/3)*mu2
vp2 = np.sqrt(M2/rho2)
vs2 = np.sqrt(mu2/rho2)

print('vp=%.2f'%vp2, 'vs=%.2f'%vs2)

>kerangka proses di atas adalah konsep fluid replacement gassmann

#### Basic Solid Phase

pemodelan rock physics akan melibatkan campuran litologi utama, umumnya dimodelkan sebagai dua elemn dasar seperti sand - shale atau calcite-shale, dsb. Campuran litologi memberikan effective modulus yang merupakan pendekatan volume rata-rata dari kedua kontituen, di representasikan dalam tiga model komposit, Voigt, Reuss dan Hill yang akan sering digunakan dalam gassmann:
<br>
   
<font size=4>$K_{Voigt} = (V_{sd} \cdot K_{qz}) + (V_{sh} \cdot K_{sh})$</font>
    
volume fraksi total dari dua elemen $V_{sd} + V_{sh} = 1 $
<br><br>

<font size=4>$K_{Reuss}=$</font> <font size=5>$({\frac{V_{sd}}{K_{sd}}}+{\frac{V_{sh}}{K_{sh}}})^{-1}$</font> 
<br><br>

<font size=4>$K_{Hill} =$</font><font size=5>$\frac {(K_{Voigt} + K_{Reuss})}{2}$</font> 

hubungan ini juga berlaku untuk shear modulus $\mu$


Bulk density diekspresikan sebagai fraksi fluida pengisi pori dan densitas matriks: <br>
<font size=4>$\rho_{b} = \rho_{ma}.(1-\phi) + \rho_{fl}.\phi$</font>

dengan campuran dua elemen batuan, misalnya sand dan shale maka, densitas matriks multimineral di atas adalah volume weighted average dari dua tipe batuan:<br>
<font size=4>$\rho_{ma} = V_{sh}.\rho_{sh} + V_{sd}.\rho_{sd}$</font>, volume fraksi total dari dua elemen $V_{sd} + V_{sh} = 1 $
<br><br>


#### Basic Fluid Phase

Demikian juga dengan campuran dua fasa fluida menggunakan pendekatan yang sama, dengan asumsi konsisten pressure maka modulus fluida dari campuran fluid element minyak dan air dalam kondisi isostress adalah representasi Reuss effective fluid model sbb:

<font size=4>$K_{fluid}=$</font><font size=5>$({\frac{S_w}{K_{brine}}}+{\frac{S_{oil}}{K_{oil}}})^{-1}$</font>, dimana $S_w + S_{oil} = 1$
<br><br> tiga fasa fluida dimungkinkan dengan : <font size=5>${\frac{1}{K_{fluid}}}={\frac{S_w}{K_w}}+{\frac{S_o}{K_o}}+{\frac{S_g}{K_g}}$ </font>

#### basic patchy fluid
<br> effective moduli dengan variasi spasial dalam batuan dapat didekati dengan Voigt model:
<br> <font size=4>$K_{fluid}=(S_w.K_{brine}+S_{oil}.K_{oil})$</font>

<br> Brie et.al. mengusulkan mixing fluid secara empiris sebagai berikut:
<br> <font size=4>$K_{brie}= (K_{liquid}-K_{gas})(1-S_{gas})^e+K_{gas})$</font>, e adalah Brie's exponent=1,2,3<br>
<br> <font size=4>$K_{liquid}=$</font><font size=5>$({\frac{S_w}{K_{brine}}}+{\frac{S_{oil}}{K_{oil}}})^{-1}$</font>

#### common values
beberapa konstanta penting yang akan digunakan dalam beberapa latihan ke depan adalah sebagai berikut:


|Mineral   |bulk modulus(k)  |shear modulus (mu)   |density (rho)   |
|:-        |----  |----: |:--:  |
|Quartz    |36.6  |44 |2.65|
|Clay      |20.9  |5  |2.58|
|Calcite   |76.8  |32 |2.71|
|Dolomite  |94.9  |45 |2.87|

|fluid |bulk modulus(k)  |density (rho)   |
|:-    |---- |:--:|
|brine |2.8  |1.09|
|oil   |0.94 |0.78|
|gas   |0.06 |0.25|

__Tahap-4__
<br>Perbedaan efek bound theories (Voigt-Reuss) terhadap solid dan fluid properties

In [ ]:
rho_qz=2.65;  k_qz=37;  mu_qz=44    # mineral properties, quartz (i.e., sands)
rho_sh=2.81;  k_sh=15;  mu_sh=5     # mineral properties, clay (i.e., shales)
rho_b=1.09;   k_b=2.8               # fluid properties, brine
rho_o=0.78;   k_o=0.94              # fluid properties, oil
rho_g=0.25;   k_g=0.06              # fluid properties, gas

In [ ]:
vshk= np.linspace(0,1,10)
vsdk= 1-vshk
kvoigt = vshk*k_sh + vsdk*k_qz
kreuss = 1/(vshk/k_sh + vsdk/k_qz)
# plt.plot
plt.rcParams["figure.figsize"] = (4,4)
plt.plot(vshk,kvoigt, label="voigt", color='C1')
plt.plot(vshk,kreuss, label="reuss", color='C2')
plt.xlabel('Vsh (clay content)'); plt.ylabel('Bulk Modulus')
plt.legend(loc = 'upper right', frameon=False, fontsize=12)
plt.xlim(0.0,1.0); plt.grid(color='gray', linestyle=':')

In [ ]:
vpvr= np.linspace(2,4,100)
vshvr= np.linspace(0,1,10)
vsdvr= 1-vshvr
vp_sh=np.sqrt(k_sh/rho_sh)
vp_qz=np.sqrt(k_qz/rho_qz)
vpvoigt = vshvr*vp_sh + vsdvr*vp_qz
vpreuss = 1/(vshvr/vp_sh + vsdvr/vp_qz)
# plt.plot
plt.rcParams["figure.figsize"] = (4,4)
plt.plot(vshvr,vpvoigt, label="voigt", color='C1')
plt.plot(vshvr,vpreuss, label="reuss", color='C2')
plt.xlabel('Vsh (clay content)'); plt.ylabel('Vp (km/s)')
plt.legend(loc = 'upper right', frameon=False, fontsize=12)
plt.xlim(0.0,1.0); plt.grid(color='gray', linestyle=':')

In [ ]:
sws1= np.linspace(0,1,100)
sos1= 1-sws1 # oil and brine mixing
sgs1= 1-sws1 # gas and brine mixing
k_b_o = 1/(sos1/k_o + sws1/k_b)
k_b_g = 1/(sgs1/k_g + sws1/k_b)
# plt.plot
plt.rcParams["figure.figsize"] = (4,4)
plt.plot(sws1,k_b_o, label="oil-brine", color='g', lw=2)
plt.plot(sws1,k_b_g, label="gas-brine", color='r', lw=2)
plt.xlabel('Swt'); plt.ylabel('Bulk modulus')
plt.legend(loc = 'upper left', frameon=False, fontsize=12)
plt.xlim(0.0,1.0); plt.grid(color='gray', linestyle=':')

In [ ]:
sw_b= np.linspace(0,1,100)
b=1 # brie exponent practical range: [2,3]
patchy_oil=0.4
so_b= patchy_oil*(1-sw_b)
sg_b= 1-sw_b-so_b
k_voigt = so_b*k_o + sw_b*k_b + sg_b*k_g
k_liq = 1/(so_b/k_o + sw_b/k_b)
k_brie = (k_liq-k_g)* ((1-sg_b)**b) + k_g
k_reussb = 1/((1-sw_b)/k_o + sw_b/k_b + sg_b/k_g)
# plt.plot
plt.rcParams["figure.figsize"] = (4,4)
plt.plot(sw_b,k_voigt, label="voigt", color='C1', lw=2)
plt.plot(sw_b,k_reussb, label="reuss", color='C2', lw=2)
k_brie2 = (k_liq-k_g)* ((1-sg_b)**2) + k_g
k_brie3 = (k_liq-k_g)* ((1-sg_b)**3) + k_g
plt.plot(sw_b,k_brie, label="brie=1", color='b', lw=0.2)
plt.plot(sw_b,k_brie2, label="brie=2", color='b', lw=2, linestyle=":")
plt.plot(sw_b,k_brie3, label="brie=3", color='b', lw=2, linestyle="--")
plt.xlabel('Swt'); plt.ylabel('Bulk modulus')
plt.legend(loc = 'upper left', frameon=False, fontsize=10)
plt.xlim(0.0,1.0); plt.grid(color='gray', linestyle=':')

__Tahap-5__
<br>Langkah gassmann isotropik melalui tahap demi tahap

### isotropic gassmann fluid replacement modeling

pemahaman alurkerja dasar gassmann dalam pemodelan dan umumnya seismic QI diperlukan karena menjadi salahsatu proses wajib yang harus dilakukan. Teori biot-gassmann menunjukkan prediksi bahwa pada setiap increment pressure yang terjadi (misalnya gelombang seismik) yang melewati batuan akan menyebabkan momen peningkatan pore pressure untuk menahan eksternal pressure yang terjadi, dan secara umum menjadikan batuan lebih stiff dari sebelumya. Prediksi ini mengaitkan hubungan saturated state dengan _dry + fluid_ elemen dari batuan, atau secara umum _saturated = dry + fluid_, dan digunakan dalam memprediksi kondisi batuan tersaturasi vs batuan tidak tersaturasi (dry) serta batuan dengan saturasi berbeda. prediksi teori ini mengambil bentuk sbb:
<br><br>
<font size=5>
$\frac {k_{sat}} {k_{ma}-k_{sat}} = \frac {k_{dry}} {k_{ma}-k_{dry}} + \frac {k_{fluid}} {k_{ma}-k_{fluid}}$
</font>
<br><br>

####  Tahap-1 : $K_{dry}$ 
<br>tahapan alurkerja gassmann dibagi menjadi dua. Pertama, memprediksi kondisi batuan dry dari batuan tersaturasi:


$k_{dry} =$ <font size=5>$\frac{k_{sat}(\frac{\phi k_{ma}}{k_{fluid}} + 1-\phi) - k_{ma}}{\frac {\phi k_{ma}}{k_{fluid}} + \frac{k_{sat}}{k_{ma}} -1-\phi} $ </font>

<br>
dimana, $k_{sat}$ adalah modulus batuan tersaturasi fluida, $k_{ma}$ modulus matriks, dan $k_{fluid}$ adalah modulud fasa fluida atau campuran fluida seperti dijelaskan sebelumnya

####   Tahap-2 : $K_{sat}$ 
<br>setelah  $k_{dry}$ didapatkan, maka dapat digunakan untuk memprediksi $k_{sat}$ yang mengandung $k_{fluid}$ baru, sbb:


$k_{sat} = k_{dry} + $ <font size=5> $\frac { (1-\frac{k_{dry}}{k_{ma}})^2} { \frac{\phi}{k_{fluid}} + \frac{1-\phi}{k_{ma}} - \frac{k_{dry}}{k_{ma}^2}}$ </font>

dalam lingkup isotropik dan batuan dengan fasa fluida konvensional, shear modulus dianggap tidak berubah, atau: 

$\mu_{sat} = \mu_{fluid} = \mu_{dry}$

dari $k_{sat}$ yang baru dihitung, maka $vp_{sat}$, $vs_{sat}$, dan $\rho_{sat}$ dapat dihitung dengan hubungan-hubungan di atas.

$\rho_{sat} = (1-\phi).\rho_{ma} + \phi.\rho_{fluid baru} $

In [ ]:
"""
Replikasi Gassmann Exercise dari buku Mavko, et.al. The Rock Physics Handbook, 2nd ed., hal: 275<br>
"""
phi=0.2; kma=36
kgas=0.133; rhogas=0.336
kwater=3.013; rhowater=1.055

ksatbrine=17.6 # water saturated rock

"""
tahap pertama, menentukan kdry dari batuan yang disaturasi water
ksatbrine/(kma-ksatbrine) = kdry/(kma-kdry) + kwater/ phi*(kma-kwater)
kdry/(kma-kdry) = ksatbrine/(kma-ksatbrine) - kwater/ phi*(kma-kwater)

"""
# solve the fluid and saturated constants
A =round((ksatbrine/(kma-ksatbrine)),3)
B =round(kwater/(phi*(kma-kwater)), 3)

#solve kdry
kdry = round((((A-B)*kma)/(1+A-B)),2)

"""
tahap kedua, menghitung ksatgas dari hasil kdry dan menggunakan kgas 
ksatgas/(kma-ksatgas) = kdry/(kma-kdry) + kwater/ phi*(kma-kwater)
"""
# solve dry dan fluid constants
C = round((kdry/(kma-kdry)),3)
D = round(kgas/(phi*(kma-kgas)), 3)

# solve ksatgas
ksatgas=round((((C+D)*kma)/(1+C+D)),2)

print(A,B, kdry,"|", C, D, ksatgas)

__Tahap-6__
<br>Aplikasi basic gassmann terhadap data sumur adi02

### contoh aplikasi untuk log sumur

In [ ]:
"---buat langkah gassmann di atas sebagai fungsi yang dapat dipanggil dan dijalankan---"
def gassmann(vp1, vs1, rho1, rho_fl1, k_fl1, rho_fl2, k_fl2, k0, phi):  
    mu1 = rho1*vs1**2.
    k1 = rho1*vp1**2-(4./3.)*mu1
    kdry= (k1 * ((phi*k0)/k_fl1+1-phi)-k0) / ((phi*k0)/k_fl1+(k1/k0)-1-phi)
    k2 = kdry + (1- (kdry/k0))**2 / ( (phi/k_fl2) + ((1-phi)/k0) - (kdry/k0**2) )
    mu2 = mu1
    rho2 = rho1 + phi*(rho_fl2-rho_fl1)
    vp2 = np.sqrt(((k2+(4./3)*mu2))/rho2)
    vs2 = np.sqrt((mu2/rho2))
    return [vp2, vs2, rho2, kdry]

In [ ]:
" --- lihat Latihan-2 untuk visualisasi. Interval akan dibatasi sekitar 1700-1800m saja---"
logs=adi02
top_plot=1720
base_plot=1760
dfrm=logs[(logs.index >= top_plot) & (logs.index <= base_plot)]
z=dfrm.index
vp_i=dfrm.Vp
vs_i=dfrm.Vs
rho_i=dfrm.Rho
sw_i=dfrm.Swt
phi=dfrm.Phit
vsh=dfrm.Vsh
vsd=1-vsh

In [ ]:
fig, axs = plt.subplots(ncols=5, figsize=(10, 6), sharey=True)
ax1 = axs[0]; ax1.plot(vp_i, z, 'k', lw=0.5); ax1.set_xlim(2,4.2); 
ax1.set_xlabel("Vp", color='k', size=16); ax1.invert_yaxis()
ax2 = axs[1]; ax2.plot(vs_i, z, 'k', lw=0.5); ax2.set_xlim(1.2,3)
ax2.set_xlabel("Vs", color='k', size=16)
ax3 = axs[2]; ax3.plot(rho_i,z, 'k', lw=0.5); ax3.set_xlim(2,2.6);
ax3.set_xlabel("Rho", color='k', size=16)
ax4 = axs[3]; ax4.plot(sw_i,z, 'k', lw=0.5); ax4.set_xlim(0.,1);
ax4.set_xlabel("Sw", color='k', size=16)
ax5 = axs[4]; ax5.plot(vsh,z, 'k', lw=0.5); ax5.set_xlim(0.,1);
ax5.set_xlabel("Vsh", color='k', size=16)
for axes in axs:
    axes.yaxis.grid(True, color="gray", linestyle=':', lw=0.4)
    axes.xaxis.grid(True, color="gray", linestyle=':', lw=0.4)
    axes.xaxis.set_ticks_position('bottom')
    axes.xaxis.set_label_position('top')

In [ ]:
kvoigt=(vsh*k_sh) + (vsd*k_qz)
kreuss=1/(vsh/k_sh + vsd/k_qz)
kvrh=0.5*(kvoigt+kreuss)
k0=kvoigt
vp1=vp_i; vs1=vs_i; rho1=rho_i
rho_fl1=rho_o; rho_fl2=rho_b
k_fl1=k_o; k_2=k_b
final_sat=1 # final saturation fraction
k_fl2=1/((final_sat/k_2)+((1-final_sat)/k_fl1))
frm=gassmann(vp1, vs1, rho1, rho_fl1, k_fl1, rho_fl2, k_fl2, k0, phi)
vp_f1=frm[0]; vs_f1=frm[1]; rho_f1=frm[2]
# print(rho_f.max())

In [ ]:
fig, axs = plt.subplots(ncols=5, figsize=(10, 6), sharey=True)
ax11 = axs[0]; ax11.set_xlim(2.6,4.2) 
ax11.plot(vp_i, z, 'k', lw=0.5, label="insitu")
ax11.set_xlabel("Vp", color='k', size=16); ax11.invert_yaxis()
ax12 = axs[0]; ax12.set_xlim(2.6,4.2) 
ax12.plot(vp_f1, z, 'b', lw=0.5, label="final")
ax21 = axs[1]; ax21.set_xlim(1.2,3)
ax21.plot(vs_i, z, 'k', lw=0.5, label="insitu")
ax21.set_xlabel("Vs", color='k', size=16)
ax22 = axs[1]; ax22.set_xlim(1.2,3)
ax22.plot(vs_f1, z, 'b', lw=0.5, label="final")
ax31 = axs[2]; ax31.set_xlim(2,2.6)
ax31.plot(rho_i,z, 'k', lw=0.5, label="insitu")
ax31.set_xlabel("Rho", color='k', size=16)
ax32 = axs[2]; ax32.set_xlim(2,2.6)
ax32.plot(rho_f1, z, 'b',lw=0.5, label="final")
ax4 = axs[3]; ax4.plot(sw_i,z, 'k', lw=0.5, label="insitu");
ax4.set_xlabel("Sw", color='k', size=16)
ax5 = axs[4]; ax5.plot(vsh,z, 'k', lw=0.5, label="insitu");
ax5.set_xlabel("Vsh", color='k', size=16)
for axes in axs:
    axes.yaxis.grid(True, color="gray", linestyle=':', lw=0.4)
    axes.xaxis.grid(True, color="gray", linestyle=':', lw=0.4)
    axes.xaxis.set_ticks_position('bottom')
    axes.xaxis.set_label_position('top')
    axes.legend(loc = 'lower right', frameon=False, fontsize=10)
# plt.savefig('03agassmann_01.png', dpi=200)

### Kdry Modeling Review

Kdry modeling dapat menggunakan beberapa format empiris tergantung tren dari data itu sendiri. Beberapa piranti lunak menggunakan graphical approaches untuk mendapatkan hubungan Kdry/Kma dan porosity, misalnya dengan pendekatan sbb:
<br>
<font size=4> $\frac {k_{dry}} {k_{ma}} = - \frac{\phi}{\phi_c}$</font> + 1 
<br><br>
<font size=4> $\frac {k_{dry}} {k_{ma}} = - \frac{1}{1+c.\phi}$</font>
<br><br>
<font size=4> $\frac {k_{dry}} {k_{ma}} = exp^{-c.\phi}$</font>
<br><br>
<font size=4> $\frac {k_{dry}} {k_{ma}} = (1-\phi)^{\frac {x}{1-\phi}}$</font>

In [ ]:
kdry=frm[3]
dfkdry=pd.DataFrame(zip(kdry,k0,phi), columns=['kdry','kma','phi'])
dfkdry['kdry'].clip(lower=0)
dfkdry.kdry = np.where(dfkdry.kdry < 0, 0, dfkdry.kdry)
dfkdry.head(10)

In [ ]:
#----plot-----
f, axes = plt.subplots(figsize=(4,4))
plt.scatter(dfkdry.kdry, dfkdry.kma, s=20, facecolors='none', edgecolors='gray')
plt.xlabel("Kdry", size=14); plt.ylabel("Kma", size=14)
plt.axvline(0, color='r',linestyle="--")
plt.show()

In [ ]:
f, axes = plt.subplots(figsize=(4,4))
kdrykma=dfkdry.kdry/dfkdry.kma
plt.scatter(dfkdry.phi, kdrykma, s=20, facecolors='none', edgecolors='gray')
isofit=1/(1+7.643*dfkdry.phi)
adifit=9.65652036* np.exp(-31.02718472 *dfkdry.phi)+0.3426115
plt.plot(dfkdry.phi,isofit,'r', lw=2)
plt.plot(dfkdry.phi,adifit,'g', linestyle="--", lw=2)
plt.xlabel("$\phi$", size=14); plt.ylabel("Kdry/Kma", size=14)
plt.show()

In [ ]:
#curve fitting review
from scipy.optimize import curve_fit
from matplotlib import pyplot

def fit(x, c):
    return 1/(1+c*x)
# curve fit
popt,_ = curve_fit(fit, dfkdry.phi, kdrykma)
print(popt)

In [ ]:
# adi fit
def adifit(x, a, b, c):
    return a * np.exp(-b * x)+c
popt, _ = curve_fit(adifit, dfkdry.phi, kdrykma)
print(popt)

In [ ]:
# gassmann with kdry model
mu1 = rho1*vs1**2.
k1 = rho1*vp1**2-(4./3.)*mu1
"---use kdry modeling---"
kdrymodel= (k0/(1+7.64*phi))
"-----------------------"
k2model = kdrymodel + (1- (kdrymodel/k0))**2 / ( (phi/k_fl2) + ((1-phi)/k0) - (kdrymodel/k0**2) )
mu2 = mu1
rho2model = rho1 + phi*(rho_fl2-rho_fl1)
vp2model = np.sqrt(((k2model+(4./3)*mu2))/rho2model)
vs2model = np.sqrt((mu2/rho2model))
print(len(vp2model))

In [ ]:
# gassmann with alternative kdry model
"---use kdry modeling---"
kdryadimodel= k0*(9.65652036 * np.exp(-31.02718472 *phi) + 0.3426115)
"-----------------------"
k2adimodel = kdryadimodel + (1- (kdryadimodel/k0))**2 / ( (phi/k_fl2) + ((1-phi)/k0) - (kdryadimodel/k0**2) )
mu2 = mu1
rho2adimodel = rho1 + phi*(rho_fl2-rho_fl1)
vp2adimodel = np.sqrt(((k2adimodel+(4./3)*mu2))/rho2adimodel)
vs2adimodel = np.sqrt((mu2/rho2adimodel))

print(len(vp2adimodel))

In [ ]:
vp_f1=frm[0]; vs_f1=frm[1]; rho_f1=frm[2]
f, axes = plt.subplots(figsize=(4,4))
plt.scatter(vp2model, vp_f1, s=40, facecolors='none', edgecolors='r')
plt.scatter(vp2adimodel, vp_f1, s=20, facecolors='g', edgecolors='g')
plt.xlabel("vp2model", size=14); plt.ylabel("vp_gassmann", size=14)
plt.plot(vp_f1,vp_f1,'k')
plt.grid(linestyle=":", color="gray")
plt.show()

# HOMEWORK

fluid replacement menggunakan isotropic gassmann mempunyai asumsi dan limitasi, karena itu harus dibatasi pada reservoir boundaries seperti: depth interval, sand interval, higher porosity interval dan sebagainya.
<br><br> Latihan dapat dilanjutkan dengan melakukan ekplorasi opsi membatasi depth interval atau sand interval saja dengan menggunakan vsh cut off (misalnya hanya dilakukan pada depth interval 1720-1740m dan vsh <=0.45 saja)
<br><br> Gunakan pilihan salahsatu cara di bawah atau temukan cara sendiri untuk membatasi interval gassmann, kemudian plot overlay hasilnya dibandingkan dengan gambar di atas,... _good luck !_

In [ ]:
import warnings
warnings.filterwarnings("ignore")

"----------contoh menggunakan filtering/mask pada dataframe------"

# membuat kolom baru temporary yang tidak perlu disimpan tetapi mempunyai array konsisten agar dapat diplot
df_baru=dfrm
df_baru.loc[:,'vp_repl']=vp_i
mask = (df_baru.index>=1720) & (df_baru.index<=1740)
vp_replaced=df_baru.Vp; vs_replaced=df_baru.Vs; rho_replaced=df_baru.Rho
replace_by_brine=gassmann(vp_replaced, vs_replaced, rho_replaced, rho_fl1, k_fl1, rho_fl2, k_fl2, k0, phi)
vp_f2=replace_by_brine[0]
df_baru.loc[mask,'vp_repl']=round(vp_f2,2)
print(df_baru)
# df_baru.drop(['vp_repl'], axis=1, inplace=True)

___________________________

In [ ]:
import warnings
warnings.filterwarnings("ignore")

" ---------contoh penggunaan pembatasan vsh--------"
df_cut=dfrm
mask2 = (df_cut.index>=1720) & (df_cut.index<=1740)
df_cut.loc[:,'vp_cutoff']=vp_i
vp3=df_cut.Vp[mask2]; vs3=df_cut.Vs[mask2]; rho3=df_cut.Rho[mask2]
vshcut=df_cut.Vsh[mask2]
for row in vshcut[:]:
    if row <= 0.4:
        frm3=gassmann(vp3, vs3, rho3, rho_fl1, k_fl1, rho_fl2, k_fl2, k0, phi)
        vp_f3=frm3[0]
    else:
        vp_f3=vp_i
df_cut.loc[mask2,'vp_cutoff']=round(vp_f3,2)
print(df_cut)
# df_cut.drop(['vp_repl', 'vp_cutoff'], axis=1, inplace=True)

<h1><center>-Akhir Program Latihan-3a-<center></h1>